In [1]:
import pandas as pd
import numpy as np
from random import randint, choice
import seaborn as sns
from scipy.sparse import csgraph
import random
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
def DTWDistance(s1, s2, data_format="pandas"):
    DTW={}
    
    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            if data_format=="pandas":
                dist= (s1.iloc[i]-s2.iloc[j])**2
            else:
                dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])

    return np.sqrt(DTW[len(s1)-1, len(s2)-1])

In [3]:
def put_weights_measure(graph, time_series, method=0, data_format="pandas"):
    new_graph = graph
    for node in range(graph.shape[0]):
        for other_node in range(graph.shape[0]):
            if node == other_node:
                pass
            elif graph[node][other_node] == 0:
                pass
            else:
                if data_format =="pandas":
                    ts1 = time_series.iloc[:, node]
                    ts2 = time_series.iloc[:,other_node]
                else:
                    ts1 = time_series[node]
                    ts2 = time_series[other_node]
                if method == 0:
                    try:
                        new_graph[node][other_node] = 1 /DTWDistance(ts1, ts2, data_format="pandas")
                    except:
                        print(ts1, ts2)
                        break
                elif method == 1:
                    new_graph[node][other_node] = np.exp(-DTWDistance(ts1, ts2, data_format="pandas"))
                else:
                    pass
    return new_graph

In [4]:
def make_consensus_matrix(clustering_matrix):
    M = np.zeros([clustering_matrix.shape[1], clustering_matrix.shape[1]])
    for i in range(clustering_matrix.shape[1]):
        for j in range(clustering_matrix.shape[1]):
            i_j = 0
            for k in range(clustering_matrix.shape[0]):
                if clustering_matrix[k,i] == clustering_matrix[k, j]:
                    i_j+=1
                else:
                    pass
            M[i, j] = i_j / clustering_matrix.shape[0]
    return M

In [5]:
def consensus(graph, time_series, n_clusters, method=0, data_format="pandas", show=False):
#     n_days = time_series.shape[0]//24
#     n_days = time_series.shape[1]
    n_nodes = graph.shape[0]
    clustering_matrix = np.array([])
    if data_format=="pandas":
        n_days = time_series.shape[0]//24
        for i in range(n_days):
            if show == True:
                if i % 30 == 0:
                    print(i)
            if method == 1:
                time_labels = time_clustering(time_series.iloc[i*24:24*(i+1), 1:])
                day_graph = put_weights(time_labels, graph)
            else:
                day_graph = put_weights_measure(graph, time_series.iloc[i*24:24*(i+1), 1:], method=0, data_format="pandas")

    #         laplacian = csgraph.laplacian(day_graph)
            graph_clustering = SpectralClustering(n_clusters=n_clusters,
                random_state=0, affinity='precomputed').fit(day_graph)

            labels = graph_clustering.labels_
    #         print(labels)
            clustering_matrix =  np.append(clustering_matrix, labels)
    if data_format=="numpy":
        n_days = time_series.shape[1]
        for day in range(n_days):
    #         print(time_labels)
            if method == 0:
                time_labels = time_clustering(time_series[:,day,:])
                day_graph = put_weights(time_labels, graph)
            else:
                day_graph = put_weights_measure(graph, time_series[:,day,:], method=0)

    #         laplacian = csgraph.laplacian(day_graph)
            graph_clustering = SpectralClustering(n_clusters=n_clusters,
                random_state=0, affinity='precomputed').fit(day_graph)

            labels = graph_clustering.labels_
    #         print(labels)
            clustering_matrix =  np.append(clustering_matrix, labels)
    
    clustering_matrix = clustering_matrix.reshape(n_days, n_nodes)
#     clustering_matrix = clustering_matrix.reshape(n_nodes, n_days)
#     print(n_days, "dff", n_nodes)
#     print(clustering_matrix)
    clustering_matrix.resize(n_days, n_nodes)
    
    consensus_matrix = make_consensus_matrix(clustering_matrix)
    
    return consensus_matrix